<a href="https://colab.research.google.com/github/katzdavid/EDA-Fall2022/blob/main/A3_EnergyManagementSystem_DRAFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ENV590.05 - Econ of Modern Power Systems - A3 - Residential PV + battery system management model**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

## Installing and Running Pyomo on Google Colab

To import/install a library that's not in Colaboratory by default, you can use !pip install. This needs to be done at the begining of you notebook. And you only need to run it once at the start of each Colab session.

In [ ]:
!pip install pyomo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 9.1 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 


## Installing optimization solver

Pyomo does not include any optimization solvers. Therefore, you will need to install third-party solvers to solve optimization models built with Pyomo. There are several solver options, for this class we will use glpk.
We'll install glpk using apt-get. 

In [ ]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123934 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

## Import Pyomo and solver

The first step in any Pyomo project is to import relevant components of the Pyomo library. This can be done with the following python statement 'from pyomo.environ import *'. \\

We use the * symbol to elimate the need of using the expression pyomo.environ every time we need to use a pyomo function. \\

You also need to load the solver. The Pyomo libary includes a SolverFactory() class used to specify a solver. Here we will use
**glpk** which works for linear problems, put **cbc** from coin OR could be used for nonlinear applications. 

In [ ]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('glpk')

## Importing Case Study data set


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
storage = pd.read_csv("Pdata.csv")
storage.columns = ["Time", "P_PV", "PLoad", "C_t"] #P_PV_t = PV generation, C_t = electricity rate
storage.head(24)

,Time,P_PV,PLoad,C_t
0,"Sep 18, 12:00 am",0.000000,2.05,0.099964
1,"Sep 18, 1:00 am",0.000000,0.32,0.099964
2,"Sep 18, 2:00 am",0.000000,1.72,0.099964
3,"Sep 18, 3:00 am",0.000000,0.34,0.099964
4,"Sep 18, 4:00 am",0.000000,1.58,0.099964
5,"Sep 18, 5:00 am",0.000000,0.34,0.099964
6,"Sep 18, 6:00 am",0.230828,0.83,0.099964
7,"Sep 18, 7:00 am",1.532470,1.55,0.099964
8,"Sep 18, 8:00 am",3.199970,0.51,0.099964
9,"Sep 18, 9:00 am",4.539360,1.98,0.099964


## Approach 2: Create Model with vectors and numpy

In [ ]:
#Creating model
model = ConcreteModel()

In [ ]:
T=range(24)  #set of hours
P_PV = storage.loc[:,"P_PV"]
P_load = storage.loc[:,"PLoad"]
C = storage.loc[:,"C_t"]
SOC_0 = 0.2
Eff = 0.92
StoCap = 4

In [ ]:
model.Pgrid = Var(T, domain = NonNegativeReals)  #Pgrid
model.Pch = Var(T, domain = NonNegativeReals, bounds = (0,3))    #Pcharge
model.Pdisch = Var(T, domain = NonNegativeReals, bounds = (0,3)) #Pdischarge
model.Pslack = Var(T, domain = NonNegativeReals) #Pslack
model.SOC = Var(T, domain = NonNegativeReals, bounds = (0.2,0.8))  #State of charge

In [ ]:
#Adding objective function


In [ ]:
#Adding constraints
model.constraints = ConstraintList()

for t in T:
  #power balance

  #charge balance
  if (t==0):

  else:
    

  #storage only charging from PV  

  #storage only discharging to supply house demand/load


In [ ]:
#Solve Model
opt.solve(model)

print("Cost =", model.cost_func())
print("\n")

Since we didn't go over how to generate plots in Python, I will provide the code that will get the decision variables value, store in a data frame and then generate plots to visualize the results.

Let's start by creating a data frame with the value of all decision variables for the 24 hours of the day. 

In [ ]:
sys_op_sch = pd.DataFrame(np.zeros((24,6)))

for t in T:
  sys_op_sch.iloc[t,0] = t+1
  sys_op_sch.iloc[t,1] =  model.Pgrid[t].value
  sys_op_sch.iloc[t,2] =  model.Pch[t].value
  sys_op_sch.iloc[t,3] =  model.Pdisch[t].value
  sys_op_sch.iloc[t,4] =  model.Pslack[t].value
  sys_op_sch.iloc[t,5] =  model.SOC[t].value

sys_op_sch = sys_op_sch.rename(columns={0:"Hour",1:"Pgrid",2:"Pcharge",3:"Pdischarge",4:"Pslack",5:"SOC"})
print(sys_op_sch)


The ggplot will automatically plot all columns in the same graph and add a legend as long as you gather the data frame as shown in the code below. Note that we just stacked the columns.

In [ ]:
sys_op_sch_gather=sys_op_sch.melt(id_vars=["Hour"])
sys_op_sch_gather.head(30)

Now we have a data frame ready to be used by ggplot. All we have to do is create the plot. 

In [ ]:
#Create plot with ggplot from library plotnine
from plotnine import ggplot, aes, geom_line
(
    ggplot(sys_op_sch_gather,aes(x="Hour",y="value",color="variable"))  
     + geom_line()
)
